In [ ]:
import os
import json
import re
import cv2
import math
import random
import shutil
import osmnx as ox
from datetime import datetime

from image_downloading import download_image

Set the Parameters for downloading images

Downloading Satellite Images adpated from code created by andolg at https://github.com/andolg/satellite-imagery-downloader (This includes image_downloading.py imported above)

OSMNX created by Geoff Boeing. 

Boeing, G. 2017. "OSMnx: New Methods for Acquiring, Constructing, Analyzing, and Visualizing Complex Street Networks." Computers, Environment and Urban Systems 65, 126-139.

In [ ]:
prefs = {
        'url': 'https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        'tile_size': 256,
        'tile_format': 'jpg',
        'dir': 'images',
        'headers': {
            'cache-control': 'max-age=0',
            'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'document',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-site': 'none',
            'sec-fetch-user': '?1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
        },
        'tl': '',
        'br': '',
        'zoom': '15'
    }

Set the Parameters for the Images

In [ ]:
startingCoordinates = [40.69,-114.23]
filename = 'FullTest' #Just the name of the file, do not include file extension

Create Helper Functions for image handling

In [ ]:
def get_image(coordArr, width):
    img = download_image(coordArr[0],coordArr[1],coordArr[0]-width,coordArr[1]+width, int(prefs['zoom']), prefs['url'], prefs['headers'], prefs['tile_size'], 3)
    
    return img

def download_OSM_network(coordArr, width, filename):
    G = ox.graph_from_bbox(coordArr[0],coordArr[0]-width,coordArr[1]+width,coordArr[1], network_type="all",truncate_by_edge = "false")
    #fig, ax1 = ox.plot_graph(G, node_size=0)
    #fig.savefig("images\\" + filename + ".png")
    ox.save_graph_geopackage(G, filepath='images\\' + filename + '.gpkg')
    
def Tile_Image(img,tile_width,name):
    img_shape = img.shape
    tile_size = (tile_width,tile_width)
    offset = (tile_width,tile_width)
    
    for i in range(int(math.ceil(img_shape[0]/(offset[1] * 1.0)))):
        for j in range(int(math.ceil(img_shape[1]/(offset[0] * 1.0)))):
            crop = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
            
            cv2.imwrite("images\\tiles\\" + name + str(i) + "_" + str(j) + ".png", crop) 

Download Satellite Image 

In [ ]:
result = get_image(startingCoordinates,0.1)

cv2.imwrite('images\ImageOutput' + filename + '.jpg', result)

Download OpenStreetMap Road Network

In [ ]:
download_OSM_network(startingCoordinates,0.1,filename)

At this point, Stop running this code. Open up QGIS and import the Satellite image and the Geopackage created above. They will need to be georeferenced together in QGIS, clipped
and each layer exported as an Image. Once that is done. Proceed with Tiling code below.

In QGIS: Add the Road Network Map from OpenStreetMap

To Import and Georeference the Satellite Image: Click Layer and Select Georeferencer. Once in the Georeferencer Window, select Open Raster and Import your Satellite Image. To add Ground Control Points to georeference the image, click on a point near the edge of the image that you can easily find the Latitude and longitude for, such as a place where two roads intersect or another obvious feature. Use Google Maps or another source to find the Lat/Long for that point and put those values into the X and Y fields in the Enter Map Coordinates Box. Click Okay when done.

Repeat adding Ground Control points at least six times, once in each corner, and once in the middle of the longest sides of the image, staying as close to the edeg of the image as possible.

After adding the Ground Control Points, click the Green Arrow button to set the settings for the Georeference, set Transformation Tyoe to Polynomial !, and Resampling Method to Cubic. Choose a name for an output file and click Ok. Click the Green Arrow again to import the Georeferenced file into the QGIS project.

#Select Layer -> Create Layer -> New Shapefile Layer in the Menu. Give it a name, and select Polygon for Geometry Type. Click Ok.

#Select the Layer in the Layers table. Click the Add Polygon Feature button in the menu bar, and add points as close to the corners of your satellite image as you can. Click the Add Polygon Feature button #again to add the Polygon to the layer.

#Select Vector -> Geoprocessing -> Clip in the menu. Selct the Input layer as the Edge layer of the Road Network, select the Clip layer you just created as your clipping layer. 

Right click on the Edges Layer and selct Symbology. change the line color to Black, and the width of the line to match the roads in the Satellite image (around 2-3 pixels should work).

Export Images: Turn off every layer except your Satellite Image. Click on Project -> Import/Export -> Export Map to Image. Under Calculate From Select Layer and select your Satellite Image layer. Change the Scale to 1:13466, and Resolution to 150 DPI. Click Save. Name your File. Click Save.

Turn off your satellite layer, and turn on the Edges layer. Click on Project -> Import/Export -> Export Map to Image. Under Calculate From Select Layer and select your Satellite Image layer. Change the Scale to 1:13466, and Resolution to 150 DPI. Click Save. Name your File. Click Save.


Import the Images you created in QGIS

In [ ]:
satelliteImageFilename = 'SatelliteNetwork.png'
roadNetworkFilename = 'RoadNetwork.png'

In [ ]:
img = cv2.imread('images\\' + satelliteImageFilename)
img2 = cv2.imread('images\\' + roadNetworkFilename)

Tile the images to create a training dataset

In [ ]:
Tile_Image(img,100,"Satellite")
Tile_Image(img2,100,"RoadMask")

Sort Images into Train/Validate/Test folders

In [ ]:
file_list = os.listdir('images\\Tiles')
roadImages = file_list[0:int(len(file_list)/2)]
satImages = file_list[int(len(file_list)/2):int(len(file_list))]
keys = list(range(0,len(satImages)))
random.shuffle(keys)

Split Data Set into 70% Training, 20% Validation, 10% Test

In [ ]:
trainingSplit = int(len(satImages)*.7)
validationSplit = int(len(satImages)*.9)

In [ ]:
for key in keys[validationSplit:len(satImages)-1]:
    shutil.copyfile('images\\Tiles\\' + roadImages[key], 'images\\Dataset\\Test\\' + roadImages[key])
    shutil.copyfile('images\\Tiles\\' + satImages[key], 'images\\Dataset\\Test\\' + satImages[key])
    
for key in keys[trainingSplit:validationSplit]:
    shutil.copyfile('images\\Tiles\\' + roadImages[key], 'images\\Dataset\\Validate\\' + roadImages[key])
    shutil.copyfile('images\\Tiles\\' + satImages[key], 'images\\Dataset\\Validate\\' + satImages[key])
    
for key in keys[0:trainingSplit]:
    shutil.copyfile('images\\Tiles\\' + roadImages[key], 'images\\Dataset\\Train\\' + roadImages[key])
    shutil.copyfile('images\\Tiles\\' + satImages[key], 'images\\Dataset\\Train\\' + satImages[key])